# Student Module Documentation

This notebook documents the Student-related models in the `apps.student` module.

## Models Overview

### 1. Student
Represents a student entity with personal and contact information.
- `registration_number`: Unique student registration number (BigInteger).
- `roll_number`: Unique roll number (String).
- `first_name`, `middle_name`, `last_name`: Student's name components.
- `date_of_birth`: Student's date of birth.
- `gender`: Gender of the student.
- `email`: Unique email address.
- `phone_number`: Contact phone number.
- `address`, `city`, `state`, `postal_code`, `country`: Address details.
- `enrollment_date`: Date when the student enrolled.
- `is_active`: Boolean flag indicating if the student is currently active.
- `created_at`, `updated_at`: Timestamps for record creation and updates.

### 2. Guardian
Represents a guardian/parent of one or more students.
- `student`: ManyToMany relationship with Student (one guardian can have multiple students).
- `first_name`, `last_name`: Guardian's name.
- `relationship`: Relationship to student (e.g., Father, Mother, Guardian).
- `phone_number`: Contact phone number.
- `email`: Email address.
- `address`: Residential address.
- `created_at`, `updated_at`: Timestamps for record tracking.

### 3. Enrollment
Represents a student's current enrollment status in a program.
- `student`: OneToOne relationship with Student (each student has one current enrollment).
- `program`: Name of the program enrolled in.
- `semester`: Current semester.
- `enrollment_date`: Date of enrollment.
- `status`: Current enrollment status with choices:
  - 'enrolled': Currently enrolled
  - 'completed': Successfully completed
  - 'dropped': Dropped out
  - 'repeated': Repeating the program/semester
- `created_at`, `updated_at`: Timestamps for record tracking.

### 4. EnrollmentHistory
Tracks historical changes to enrollment status.
- `enrollment`: ForeignKey to Enrollment.
- `student`: ForeignKey to Student.
- `semester`: Semester when the change occurred.
- `previous_status`: Previous enrollment status.
- `new_status`: New enrollment status.
- `changed_at`: Timestamp of status change.
- `changed_by`: Identifier of who made the change (user or system).

### 5. AcademicRecord
Stores semester-wise academic performance records.
- `student`: ForeignKey to Student (one student can have multiple academic records).
- `semester`: Semester identifier.
- `gpa`: Grade Point Average (decimal with 2 decimal places).
- `total_credits`: Total credits earned in the semester.
- `remarks`: Optional remarks or notes.
- `created_at`, `updated_at`: Timestamps for record tracking.

### 6. SubjectResult
Stores individual subject results for students.
- `student`: ForeignKey to Student.
- `subject_name`: Name of the subject.
- `marks_obtained`: Marks scored (optional).
- `maximum_marks`: Maximum marks possible (optional).
- `grade`: Letter grade with choices (A+, A, B+, B, C+, C, D, F, I-Incomplete, W-Withdrawn).
- `semester`: Semester identifier.
- `attempt_type`: Type of attempt with choices:
  - 'regular': Regular examination
  - 're_exam': Re-examination
- `created_at`, `updated_at`: Timestamps for record tracking.

### 7. Document
Stores documents associated with students.
- `student`: ForeignKey to Student.
- `document_type`: Type/category of document.
- `file`: FileField for document upload (stored in 'student_documents/').
- `uploaded_at`: Timestamp when document was uploaded.

## Usage Examples

```python
from apps.student.models import (
    Student, Guardian, Enrollment, EnrollmentHistory,
    AcademicRecord, SubjectResult, Document
)
from datetime import date
from decimal import Decimal

# Create a Student
student = Student.objects.create(
    registration_number=202401001,
    roll_number="CS2024001",
    first_name="John",
    middle_name="Michael",
    last_name="Doe",
    date_of_birth=date(2006, 5, 15),
    gender="Male",
    email="john.doe@example.com",
    phone_number="1234567890",
    address="123 Main Street",
    city="New York",
    state="NY",
    postal_code="10001",
    country="USA",
    enrollment_date=date(2024, 9, 1),
    is_active=True
)

# Create a Guardian
guardian = Guardian.objects.create(
    first_name="Robert",
    last_name="Doe",
    relationship="Father",
    phone_number="0987654321",
    email="robert.doe@example.com",
    address="123 Main Street"
)
guardian.student.add(student)

# Create an Enrollment
enrollment = Enrollment.objects.create(
    student=student,
    program="Computer Science",
    semester="Fall 2024",
    enrollment_date=date(2024, 9, 1),
    status=Enrollment.ENROLLMENT_STATUS_ENROLLED
)

# Add Subject Results
result = SubjectResult.objects.create(
    student=student,
    subject_name="Data Structures",
    marks_obtained=Decimal('85.50'),
    maximum_marks=Decimal('100.00'),
    grade='A',
    semester="Fall 2024",
    attempt_type=SubjectResult.ATTEMPT_CHOICE_REGULAR
)

# Create Academic Record
academic_record = AcademicRecord.objects.create(
    student=student,
    semester="Fall 2024",
    gpa=Decimal('3.75'),
    total_credits=18,
    remarks="Excellent performance"
)

# Track Enrollment Status Change
enrollment.status = Enrollment.ENROLLMENT_STATUS_COMPLETED
enrollment.save()

EnrollmentHistory.objects.create(
    enrollment=enrollment,
    student=student,
    semester="Fall 2024",
    previous_status=Enrollment.ENROLLMENT_STATUS_ENROLLED,
    new_status=Enrollment.ENROLLMENT_STATUS_COMPLETED,
    changed_by="admin_user"
)
```

## Logic and Validation Rules

1. **Student Uniqueness**: Both `registration_number` and `roll_number` must be unique. Email must also be unique.
2. **Enrollment Relationship**: Each student can have only ONE current enrollment (OneToOne relationship).
3. **Guardian Flexibility**: One guardian can be associated with multiple students (ManyToMany relationship).
4. **Enrollment Status Tracking**: Any change to enrollment status should create an EnrollmentHistory record for audit trail.
5. **Grade Options**: SubjectResult supports both marks-based and grade-based evaluation. Either can be null/blank.
6. **Attempt Types**: SubjectResults can be marked as 'regular' or 're_exam' to track re-examination attempts.
7. **Academic Record**: One student can have multiple academic records (one per semester).
8. **Document Storage**: Student documents are stored in 'student_documents/' directory.

## Recommendations and Improvements

### Critical Issues to Address:

#### 1. **Enrollment Model Design Flaw**
**Issue**: Using `OneToOneField` for student enrollment is problematic. Students typically enroll in multiple programs/semesters over time.

**Recommendation**:
```python
# Change from:
student = models.OneToOneField(Student, ...)

# To:
student = models.ForeignKey(Student, on_delete=models.CASCADE, related_name='enrollments')

# Add unique constraint:
class Meta:
    unique_together = [['student', 'program', 'semester']]
```

#### 2. **Missing Validation in Models**
**Issue**: No validation logic in model's `clean()` method.

**Recommendations**:
- Add date validation (enrollment_date should not be in future for Student)
- Validate date_of_birth (student should be at least a certain age)
- Validate marks_obtained <= maximum_marks in SubjectResult
- Validate GPA range (0.00 to 4.00 typically)
- Validate that enrollment_date in Enrollment matches student's enrollment_date logic

```python
# Example for SubjectResult:
def clean(self):
    from django.core.exceptions import ValidationError
    if self.marks_obtained and self.maximum_marks:
        if self.marks_obtained > self.maximum_marks:
            raise ValidationError('Marks obtained cannot exceed maximum marks')
    if self.marks_obtained and self.marks_obtained < 0:
        raise ValidationError('Marks cannot be negative')
```

#### 3. **Inconsistent Import Pattern**
**Issue**: In `academic_record.py`, the import is `from . import student` then uses `student.Student`, while other files use `from . import Student`.

**Recommendation**: Standardize to `from . import Student` for consistency.

#### 4. **Missing Index Optimization**
**Issue**: No database indexes defined for frequently queried fields.

**Recommendations**:
```python
class Student(models.Model):
    # Add indexes for frequently searched fields
    class Meta:
        db_table = 'students'
        ordering = ['first_name', 'last_name']
        indexes = [
            models.Index(fields=['email']),
            models.Index(fields=['registration_number']),
            models.Index(fields=['is_active', 'enrollment_date']),
        ]
```

#### 5. **Enrollment Status Change Automation**
**Issue**: Creating EnrollmentHistory is manual and can be forgotten.

**Recommendation**: Use Django signals to automatically create history records:
```python
# In signals.py
from django.db.models.signals import pre_save
from django.dispatch import receiver

@receiver(pre_save, sender=Enrollment)
def track_enrollment_changes(sender, instance, **kwargs):
    if instance.pk:  # Only for updates
        old_enrollment = Enrollment.objects.get(pk=instance.pk)
        if old_enrollment.status != instance.status:
            EnrollmentHistory.objects.create(
                enrollment=instance,
                student=instance.student,
                semester=instance.semester,
                previous_status=old_enrollment.status,
                new_status=instance.status,
                changed_by='system'  # Can be enhanced to track actual user
            )
```

#### 6. **Missing Soft Delete**
**Issue**: No soft delete mechanism. Deleting a student would cascade delete all related data.

**Recommendation**: Consider adding soft delete functionality:
```python
class Student(models.Model):
    is_active = models.BooleanField(default=True)
    is_deleted = models.BooleanField(default=False)  # Add this
    deleted_at = models.DateTimeField(null=True, blank=True)  # Add this
```

#### 7. **Guardian Address Redundancy**
**Issue**: Guardian has only one address, but may need to store different addresses for different students.

**Recommendation**: Consider if guardian address is needed, or if it should be student-specific.

#### 8. **Missing Credit Hours in SubjectResult**
**Issue**: SubjectResult doesn't track credit hours, making GPA calculation difficult.

**Recommendation**:
```python
class SubjectResult(models.Model):
    # Add:
    credit_hours = models.IntegerField(default=3)
```
